# ENTREGA 3 -----> Carlos Martin Sanz

In [40]:
import numpy as np
from scipy.stats import entropy
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export_text
from re import findall, finditer

In [41]:
# Creo la tabla de los apuntes de la asignatura

tablaAP = pd.DataFrame(columns = ['Presion','Clase'], index = np.arange(6))
tablaAP.Presion = [40, 48, 60, 72, 80, 90]
tablaAP.Clase = ['+', '+', '-', '-', '-', '+']
tablaAP

,Presion,Clase
0,40,+
1,48,+
2,60,-
3,72,-
4,80,-
5,90,+


In [42]:
lista = []
for i in tablaAP.Clase:
    if i == '+':
        lista.append(1)
    else:
        lista.append(0)
lista

[1, 1, 0, 0, 0, 1]

In [43]:
H = entropy(tablaAP.Clase.value_counts(normalize = True), base = 2)
H

1.0

In [44]:
tablaAP1 = tablaAP
tablaAP1.Presion = np.digitize(tablaAP.Presion, bins = [(48 + 60) / 2])
tablaAP1

,Presion,Clase
0,0,+
1,0,+
2,1,-
3,1,-
4,1,-
5,1,+


In [45]:
l1 = len(tablaAP1)
GInfor1 = H
for v in tablaAP1.Presion.unique():
    Class1 = tablaAP1.loc[tablaAP.Presion == v]
    GInfor1 -= (len(Class1)/l1)*entropy(Class1.iloc[:, -1].value_counts(normalize=True), base = 2)
GInfor1

0.4591479170272448

In [46]:
tablaAP.Presion = [40, 48, 60, 72, 80, 90]
tablaAP

,Presion,Clase
0,40,+
1,48,+
2,60,-
3,72,-
4,80,-
5,90,+


In [47]:
tablaAP2 = tablaAP
tablaAP2.Presion = np.digitize(tablaAP.Presion, bins = [(80 + 90) / 2])
tablaAP2

,Presion,Clase
0,0,+
1,0,+
2,0,-
3,0,-
4,0,-
5,1,+


In [48]:
l2 = len(tablaAP2)
GInfor2 = H
for v in tablaAP2.Presion.unique():
    Class2 = tablaAP2.loc[tablaAP2.Presion == v]
    GInfor2 -= (len(Class2) / l2) * entropy(Class2.iloc[:, -1].value_counts(normalize = True), base = 2)
GInfor2

0.19087450462110933

Despues de hallar GInfor1 y GInfor2 se observa que GInfor1 > GInfor2, por ello se escoge la primera cota de digitalización (Presión = 54)

# Comprobar que usando discretizacion basada en la entropia obtiene el mismo resultado

In [49]:
tablaAP.Presion = [40, 48, 60, 72, 80, 90]
tablaAP


,Presion,Clase
0,40,+
1,48,+
2,60,-
3,72,-
4,80,-
5,90,+


In [50]:
clf = DecisionTreeClassifier(max_leaf_nodes=2,criterion = 'entropy')

In [51]:
clf.fit(np.array(tablaAP['Presion']).reshape(-1, 1), tablaAP.Clase)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=2,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [52]:
txt = export_text(clf, feature_names=['Presion'])
print(txt)

|--- Presion <= 54.00
|   |--- class: +
|--- Presion >  54.00
|   |--- class: -



In [53]:
export_graphviz(decision_tree=clf, out_file='Presion.dot', class_names= tablaAP.Clase)

In [54]:
!dot Presion.dot -Tpng -o arbol_presion.png

<img src='arbol_presion.png'>

# -------------------------------------------------------------------------

# PARTE 2 (WEKA)

## Pregunta 1


La razón por la que no se puede aplicar directamente ID3, es porque existen algunos atributos como son AGE, PRE4, PRE5 que contienen valores numericos (valores continuos)

# -------------------------------------------------------------------------

## Pregunta 2

<img src='Prescindibles.png'>

Se puede saber que atributos no son necesarios o se puede prescindir de ellos en WEKA, para ello, una vez cargado el archivo tienes que ir al apartado de Select Atributes, en la parte de Attribute Evaluator seleccionar InfoGainAttributeEval, el atributo que hay que elegir para clasificarlo bien es de abajo (Risk1Yr) y le damos a start, como podemos observar en la foto de color azul existen 3 atributos con 0 en ganancia por tanto los atributos de los que se puede prescindir son PRE4 ,AGE Y PRE5

In [55]:
tablaTS = pd.read_csv('ThoraricSurgery.csv')
tablaTS

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,DGN2,2.88,2.16,PRZ1,F,F,F,T,T,OC14,F,F,F,T,F,60,F
1,DGN3,3.40,1.88,PRZ0,F,F,F,F,F,OC12,F,F,F,T,F,51,F
2,DGN3,2.76,2.08,PRZ1,F,F,F,T,F,OC11,F,F,F,T,F,59,F
3,DGN3,3.68,3.04,PRZ0,F,F,F,F,F,OC11,F,F,F,F,F,54,F
4,DGN3,2.44,0.96,PRZ2,F,T,F,T,T,OC11,F,F,F,T,F,73,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,DGN2,3.88,2.12,PRZ1,F,F,F,T,F,OC13,F,F,F,T,F,63,F
466,DGN3,3.76,3.12,PRZ0,F,F,F,F,F,OC11,F,F,F,T,F,61,F
467,DGN3,3.04,2.08,PRZ1,F,F,F,T,F,OC13,F,F,F,F,F,52,F
468,DGN3,1.96,1.68,PRZ1,F,F,F,T,T,OC12,F,F,F,T,F,79,F


In [56]:
with open('ThoraricSurgery.txt') as file:
    ficheroTxt= file.read()

In [57]:
ficheroTxt


'PRE9 = F\n|   PRE17 = F\n|   |   DGN = DGN2\n|   |   |   PRE30 = T\n|   |   |   |   PRE10 = T\n|   |   |   |   |   PRE6 = PRZ1\n|   |   |   |   |   |   PRE14 = OC14: F (3.0)\n|   |   |   |   |   |   PRE14 = OC12\n|   |   |   |   |   |   |   PRE8 = F: T (6.0/2.0)\n|   |   |   |   |   |   |   PRE8 = T: F (6.0/1.0)\n|   |   |   |   |   |   PRE14 = OC11: F (5.0/1.0)\n|   |   |   |   |   |   PRE14 = OC13: F (2.0)\n|   |   |   |   |   PRE6 = PRZ0: T (2.0)\n|   |   |   |   |   PRE6 = PRZ2: F (4.0/1.0)\n|   |   |   |   PRE10 = F: F (10.0/1.0)\n|   |   |   PRE30 = F: F (6.0)\n|   |   DGN = DGN3\n|   |   |   PRE7 = F\n|   |   |   |   PRE14 = OC14: F (7.0/3.0)\n|   |   |   |   PRE14 = OC12\n|   |   |   |   |   PRE30 = T\n|   |   |   |   |   |   PRE10 = T\n|   |   |   |   |   |   |   PRE6 = PRZ1\n|   |   |   |   |   |   |   |   PRE25 = F\n|   |   |   |   |   |   |   |   |   PRE11 = T: F (16.0/1.0)\n|   |   |   |   |   |   |   |   |   PRE11 = F\n|   |   |   |   |   |   |   |   |   |   PRE8 = F: F 

In [58]:
# Ahora hay que recorrer las columnas de la tabla csv y comprobar si está y toma valor -1 en el txt 
# que hemos obtenido de weka

for f in tablaTS.columns:
    if ficheroTxt.find(f) == -1:
        print(f, ' ---> Este atributo no se utiliza')

PRE19  ---> Este atributo no se utiliza
PRE32  ---> Este atributo no se utiliza
Risk1Yr  ---> Este atributo no se utiliza


## Arbol con weka con todos los atributos

<img src="j48_allatributes.png">

## Arbol con weka prescindiendo de PRE19 y PRE32 

<img src="arbol_j48atributoseliminados.png">

# -------------------------------------------------------------------------

## Pregunta 3

Esto es porque J48 discretiza los valores que son de importancia para el nodo por el cual se llega, 
es decir, unicamente discretiza los valores relevantes. Sin embargo el otro método que hemos usado no, por tanto,
hará un arbol menos eficiente y de peor calidad, puesto que no se centra en los valores que son de importancia
para el nodo.

# ------------------------------------------------------------------------- 

## Pregunta 4


In [59]:
for atributo in ['PRE4', 'PRE5', 'AGE']:
    
    array = []

    for pos in finditer(atributo+' '+'<= ', ficheroTxt):
        
        print (atributo+' '+'<= ', pos.start(), pos.end())
        
        if ficheroTxt[pos.end():].find(':') < ficheroTxt[pos.end():].find('\n'):
            segmento = ficheroTxt[pos.end():].find(':')
        else:
            segmento = ficheroTxt[pos.end():].find('\n')
            
        array.append(float(ficheroTxt[pos.end():pos.end()+segmento]))
        print(array[-1])
        array.sort()
    tablaTS[atributo] = [chr(65+i) for i in np.digitize(tablaTS[atributo], bins=array)]

PRE4 <=  1078 1086
2.88
PRE4 <=  1804 1812
2.66
PRE5 <=  1976 1984
2.04
AGE <=  2013 2020
70.0
AGE <=  2627 2634
65.0
AGE <=  2797 2804
62.0


In [60]:
tablaTS.AGE.unique()

array(['A', 'D', 'C', 'B'], dtype=object)

In [61]:
tablaTS

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,DGN2,C,B,PRZ1,F,F,F,T,T,OC14,F,F,F,T,F,A,F
1,DGN3,C,A,PRZ0,F,F,F,F,F,OC12,F,F,F,T,F,A,F
2,DGN3,B,B,PRZ1,F,F,F,T,F,OC11,F,F,F,T,F,A,F
3,DGN3,C,B,PRZ0,F,F,F,F,F,OC11,F,F,F,F,F,A,F
4,DGN3,A,A,PRZ2,F,T,F,T,T,OC11,F,F,F,T,F,D,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,DGN2,C,B,PRZ1,F,F,F,T,F,OC13,F,F,F,T,F,B,F
466,DGN3,C,B,PRZ0,F,F,F,F,F,OC11,F,F,F,T,F,A,F
467,DGN3,C,B,PRZ1,F,F,F,T,F,OC13,F,F,F,F,F,A,F
468,DGN3,A,A,PRZ1,F,F,F,T,T,OC12,F,F,F,T,F,D,F


In [62]:
tablaTS.to_csv('Thoracic.Nominal.csv', index=False)

Este nuevo arbol ha obtenido mejores resultados durante la fase de test(75,6% de acierto vs 75%).Y los principales motivos por los cuales ha obtenido mejores resultados son, que los intervalos de particionamiento para los atributos numericos, han sido seleccionados de forma optima segun los resultados del algoritmo j48, y porque este arbol recoge todas las posibles combinaciones para cada caso gracias a la discretizacion que hemos realizado. 

Aunque este ultimo motivo puede producir ventajas o desventajas, en este cso ha sido beneficioso.

## Arbol con el nuevo particionamiento de los atributos numericos

<img src="arbol_nuevo.png">